In [1]:
# retinanet bounding box co_ordinates
import json
with open(r"",'r') as file:
    co_ords_dict=json.load(file)
coordinates=[]
for i in co_ords_dict:
    coordinates.append(co_ords_dict[i])
import os
image_files=r""
all_files= os.listdir(image_files)
all_files.sort( key= lambda x: int(os.path.splitext(x)[0])) 
files=[i[0] for i in all_files]
print(files)

['1', '2', '3', '4', '5']


In [ ]:
# loading models
from model import BlipImageCaptioningModel,GitBaseModel,VIT_GPT2_Model
blip=BlipImageCaptioningModel()
git=GitBaseModel()
vit_gpt2=VIT_GPT2_Model()

In [ ]:
from PIL import Image
vit_gpt_2_preds={}
for i,j in zip(all_files,coordinates):
    image=Image.open(image_files+"\\"+i)
    vit_gpt_2_preds[i]=[]
    for coordinate in j:
        cropped_image=image.crop(coordinate)
        pixel_values = vit_gpt2.feature_extractor(images=cropped_image, return_tensors="pt").pixel_values
        output_ids = vit_gpt2.model.generate(pixel_values)
        prediction = vit_gpt2.tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0]
        vit_gpt_2_preds[i].append(prediction)
blip_preds={}
for i,j in zip(all_files,coordinates):
    image=Image.open(image_files+"\\"+i)
    blip_preds[i]=[]
    for coordinate in j:
       cropped_image=image.crop(coordinate)
       inputs = blip.processor(cropped_image, return_tensors="pt")
       out = blip.model.generate(**inputs)
       prediction=blip.processor.decode(out[0], skip_special_tokens=True)
       blip_preds[i].append(prediction)

# # for GIT
git_preds={}
for i,j in zip(all_files,coordinates):
    image=Image.open(image_files+"\\"+i)
    git_preds[i]=[]
    for coordinate in j:
       cropped_image=image.crop(coordinate)
       pixel_values = git.processor(images=cropped_image, return_tensors="pt").pixel_values
       generated_ids = git.model.generate(pixel_values=pixel_values, max_length=50)
       generated_caption = git.processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
       git_preds[i].append(generated_caption)

In [1]:
import ast
import json
with open(r"",'r') as file :
    txt_data=file.read()
    parsed_dictionary=ast.literal_eval(txt_data)
with open(r"",'w') as file:
    json.dump(parsed_dictionary,file)
with open(r"",'r') as file :
    txt_data=file.read()
    parsed_dictionary=ast.literal_eval(txt_data)
with open(r"",'w') as file:
    json.dump(parsed_dictionary,file)
with open(r"",'r') as file :
    txt_data=file.read()
    parsed_dictionary=ast.literal_eval(txt_data)
with open(r"",'w') as file:
    json.dump(parsed_dictionary,file)


In [2]:
with open(r"",'r') as file:
    blip_preds=json.load(file)
with open(r"",'r') as file:
    git_preds=json.load(file)
with open(r"",'r') as file:
    vit_gpt_2_preds=json.load(file)

In [3]:
# creating graphs
print(vit_gpt_2_preds)
print(blip_preds)
print(git_preds)

{'1.jpg': ['man wearing blue and blue shirt sitting on bench ', 'Woman sitting on bench talking on cell phone ', 'a man is sitting on a bench ', 'woman sitting on the ground with a bag '], '2.jpg': ['a man is standing on the ground ', 'car is on the road near the curb ', 'white and white on the floor ', 'a red and black and red and red and red and red '], '3.jpg': ['baseball game in progress with the dugout ', 'baseball game in progress with batter up to plate ', "a baseball catcher's hat on the wall ", 'a baseball player swinging a bat ', 'part of a dark background ', 'a man wearing orange and orange is watching the game ', 'part of a dark background ', 'baseball field with grass and dirt ', 'part of a dark background ', 'a person sitting at a baseball game ', 'a man holding a bat in front of a catcher ', 'part of a dark background ', 'a person standing in the stands ', 'a person watching a game on television '], '4.jpg': ['a dog standing next to a dog ', 'a stove top oven with a blac

In [6]:
# draw images and labels for each model
from PIL import Image,ImageDraw,ImageFont
def draw_boxes_and_labels(image, coordinates, labels):
    img = Image.open(image)
    draw = ImageDraw.Draw(img)
    font = ImageFont.load_default()

    for coords, label in zip(coordinates, labels):
        x1, y1, x2, y2 = coords
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
        draw.text((x1, y1 - 10), label, fill="black", font=font)

    return img
image_files=r""
#for vit_gpt2
for i in vit_gpt_2_preds:
    image_path= image_files+"\\"+i
    vit_gpt2_image=draw_boxes_and_labels(image_path,coordinates[int(i[0])-1],vit_gpt_2_preds[i])
    vit_gpt2_image.save(f"ViT_GPT2_{i}")

#for blip
for i in blip_preds:
    image_path= image_files+"\\"+i
    blip_image=draw_boxes_and_labels(image_path,coordinates[int(i[0])-1],blip_preds[i])
    blip_image.save(f"BLIP{i}")        

# for git
for i in git_preds:
    image_path= image_files+"\\"+i
    git_image=draw_boxes_and_labels(image_path,coordinates[int(i[0])-1],git_preds[i])
    git_image.save(f"GiT{i}")        


In [ ]:
# generating graphs for each model description
from graph_generator import NLP,build_graph
nlp=NLP() 

In [5]:
# for vit_gpt2
vit_gpt2_graphlets={}
for i in vit_gpt_2_preds:
    vit_gpt2_graphlets[i]=build_graph(nlp,vit_gpt_2_preds[i])
# for blip
blip_graphlets={}
for i in blip_preds:
    blip_graphlets[i]=build_graph(nlp,blip_preds[i])
# for git
git_graphlets={}
for i in git_preds:
    git_graphlets[i]=build_graph(nlp,git_preds[i])

man wearing blue and blue shirt sitting on bench 
[1, 'man', 'man', 'NN', ['nsubj', 2, 'wearing']]
[2, 'wearing', 'wear', 'VBG', 'root']
[3, 'blue', 'blue', 'JJ', ['amod', 6, 'shirt']]
[4, 'and', 'and', 'CC', ['cc', 5, 'blue']]
[5, 'blue', 'blue', 'JJ', ['conj', 3, 'blue']]
[6, 'shirt', 'shirt', 'NN', ['obj', 2, 'wearing']]
[7, 'sitting', 'sit', 'VBG', ['acl', 6, 'shirt']]
[8, 'on', 'on', 'IN', ['case', 9, 'bench']]
[9, 'bench', 'bench', 'NN', ['obl', 7, 'sitting']]


Entities:
1 man man NN
6 shirt shirt NN
9 bench bench NN

Actions:
2 wearing wear VBG
7 sitting sit VBG

Attributes:
3 blue blue JJ
5 blue blue JJ

Relations:
man agent wear
shirt attr blue
blue conjunction blue
wear patient shirt
sit agent shirt
sit on bench



Woman sitting on bench talking on cell phone 
[1, 'Woman', 'woman', 'NN', 'root']
[2, 'sitting', 'sit', 'VBG', ['acl', 1, 'Woman']]
[3, 'on', 'on', 'IN', ['case', 4, 'bench']]
[4, 'bench', 'bench', 'NN', ['obl', 2, 'sitting']]
[5, 'talking', 'talk', 'VBG', ['advcl

In [ ]:
from graph_generator import draw_graph
# graphlet drawing for vit-gpt2 
vit_gpt2_graphs={}
for i in vit_gpt2_graphlets:
    vit_gpt2_graphs[i]=[]
    for graphlet in vit_gpt2_graphlets[i]:
        vit_gpt2_graphs[i].append(draw_graph(graphlet))

# for blip
blip_graphs={}
for i in blip_graphlets:
    blip_graphs[i]=[]
    for graphlet in blip_graphlets[i]:
        blip_graphs[i].append(draw_graph(graphlet))

# for git
git_graphs={}
for i in git_graphlets:
    git_graphs[i]=[]
    for graphlet in git_graphlets[i]:
        git_graphs[i].append(draw_graph(graphlet))

In [7]:
sum_edges=0
sum_nodes=0
for i in vit_gpt2_graphs:
    for graphlet in vit_gpt2_graphs[i]:
        sum_nodes+=graphlet.number_of_nodes()
        sum_edges+=graphlet.number_of_edges()
print(sum_nodes/26)
print(sum_edges/26)
print(sum_nodes)
print(sum_edges)

3.769230769230769
3.0
98
78


In [8]:
sum_edges=0
sum_nodes=0
for i in blip_graphs:
    for graphlet in blip_graphs[i]:
        sum_nodes+=graphlet.number_of_nodes()
        sum_edges+=graphlet.number_of_edges()
print(sum_nodes/26)
print(sum_edges/26)
print(sum_nodes)
print(sum_edges)

2.4615384615384617
1.4230769230769231
64
37


In [9]:
sum_edges=0
sum_nodes=0
for i in blip_graphs:
    for graphlet in git_graphs[i]:
        sum_nodes+=graphlet.number_of_nodes()
        sum_edges+=graphlet.number_of_edges()
print(sum_nodes/26)
print(sum_edges/26)
print(sum_nodes)
print(sum_edges)

3.1153846153846154
2.1538461538461537
81
56


In [10]:
from brute_graph_merging import merge_graphlets
sum_nodes=0
sum_edges=0
for i in vit_gpt2_graphs:
    nodes,edges=merge_graphlets(vit_gpt2_graphs[i],coordinates[int(i[0])-1])
    sum_nodes+=nodes
    sum_edges+=edges
print(f"Total sum of nodes: {sum_nodes}")
print(f"Total sum of edges: {sum_edges}")

Total sum of nodes: 94
Total sum of edges: 77


In [11]:
from brute_graph_merging import merge_graphlets
sum_nodes=0
sum_edges=0
for i in blip_graphs:
    nodes,edges=merge_graphlets(blip_graphs[i],coordinates[int(i[0])-1])
    sum_nodes+=nodes
    sum_edges+=edges
print(f"Total sum of nodes: {sum_nodes}")
print(f"Total sum of edges: {sum_edges}")

Total sum of nodes: 62
Total sum of edges: 37


In [12]:
from brute_graph_merging import merge_graphlets
sum_nodes=0
sum_edges=0
for i in git_graphs:
    nodes,edges=merge_graphlets(git_graphs[i],coordinates[int(i[0])-1])
    sum_nodes+=nodes
    sum_edges+=edges
print(f"Total sum of nodes: {sum_nodes}")
print(f"Total sum of edges: {sum_edges}")

Total sum of nodes: 77
Total sum of edges: 53


In [13]:
# enhanced parsing
from enhanced_graph_generator import NLP,build_graph,draw_graph
nlp=NLP()

2023-12-08 13:32:25 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-12-08 13:32:37 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |
| sentiment    | sstplus             |
| ner          | ontonotes           |

2023-12-08 13:32:37 INFO: Using device: cpu
2023-12-08 13:32:37 INFO: Loading: tokenize
2023-12-08 13:32:37 INFO: Loading: pos
2023-12-08 13:32:38 INFO: Loading: lemma
2023-12-08 13:32:38 INFO: Loading: constituency
2023-12-08 13:32:40 INFO: Loading: depparse
2023-12-08 13:32:43 INFO: Loading: sentiment
2023-12-08 13:32:46 INFO: Loading: ner
2023-12-08 13:3

In [14]:
vit_gpt2_graphlets={}
for i in vit_gpt_2_preds:
    vit_gpt2_graphlets[i]=build_graph(nlp,vit_gpt_2_preds[i])
# for blip
blip_graphlets={}
for i in blip_preds:
    blip_graphlets[i]=build_graph(nlp,blip_preds[i])
# for git
git_graphlets={}
for i in git_preds:
    git_graphlets[i]=build_graph(nlp,git_preds[i])

(1, 'man', 'man', 'NN', ['nsubj', 2, 'wearing'])
(2, 'wearing', 'wear', 'VBG', [])
(3, 'blue', 'blue', 'JJ', ['amod', 6, 'shirt'])
(4, 'and', 'and', 'CC', ['cc', 5, 'blue'])
(5, 'blue', 'blue', 'JJ', ['conj:and', 3, 'blue'])
(5, 'blue', 'blue', 'JJ', ['amod', 6, 'shirt'])
(6, 'shirt', 'shirt', 'NN', ['obj', 2, 'wearing'])
(7, 'sitting', 'sit', 'VBG', ['acl', 6, 'shirt'])
(8, 'on', 'on', 'IN', ['case', 9, 'bench'])
(9, 'bench', 'bench', 'NN', ['obl:on', 7, 'sitting'])


Entities:
1 man man NN
6 shirt shirt NN
9 bench bench NN

Actions:
2 wearing wear VBG
7 sitting sit VBG

Attributes:
3 blue blue JJ
5 blue blue JJ
5 blue blue JJ

Relations:
man agent wear
shirt attr blue
blue and blue
shirt attr blue
wear patient shirt
Nonetype encountered



(1, 'Woman', 'woman', 'NN', [])
(2, 'sitting', 'sit', 'VBG', ['acl', 1, 'Woman'])
(3, 'on', 'on', 'IN', ['case', 4, 'bench'])
(4, 'bench', 'bench', 'NN', ['obl:on', 2, 'sitting'])
(5, 'talking', 'talk', 'VBG', ['advcl', 2, 'sitting'])
(6, 'on', 'on

In [ ]:
vit_gpt2_graphs={}
for i in vit_gpt2_graphlets:
    vit_gpt2_graphs[i]=[]
    for graphlet in vit_gpt2_graphlets[i]:
        vit_gpt2_graphs[i].append(draw_graph(graphlet))

In [ ]:
# for blip
blip_graphs={}
for i in blip_graphlets:
    blip_graphs[i]=[]
    for graphlet in blip_graphlets[i]:
        blip_graphs[i].append(draw_graph(graphlet))

In [ ]:
# for git
git_graphs={}
for i in git_graphlets:
    git_graphs[i]=[]
    for graphlet in git_graphlets[i]:
        git_graphs[i].append(draw_graph(graphlet))

In [18]:
sum_edges=0
sum_nodes=0
for i in vit_gpt2_graphs:
    for graphlet in vit_gpt2_graphs[i]:
        sum_nodes+=graphlet.number_of_nodes()
        sum_edges+=graphlet.number_of_edges()
print(sum_nodes/26)
print(sum_edges/26)
print(sum_nodes)
print(sum_edges)

3.769230769230769
3.3461538461538463
98
87


In [19]:
sum_edges=0
sum_nodes=0
for i in blip_graphs:
    for graphlet in blip_graphs[i]:
        sum_nodes+=graphlet.number_of_nodes()
        sum_edges+=graphlet.number_of_edges()
print(sum_nodes/26)
print(sum_edges/26)
print(sum_nodes)
print(sum_edges)

2.4615384615384617
1.9230769230769231
64
50


In [20]:
sum_edges=0
sum_nodes=0
for i in git_graphs:
    for graphlet in git_graphs[i]:
        sum_nodes+=graphlet.number_of_nodes()
        sum_edges+=graphlet.number_of_edges()
print(sum_nodes/26)
print(sum_edges/26)
print(sum_nodes)
print(sum_edges)

3.076923076923077
2.230769230769231
80
58


In [21]:
from brute_graph_merging import merge_graphlets
sum_nodes=0
sum_edges=0
for i in vit_gpt2_graphs:
    nodes,edges=merge_graphlets(vit_gpt2_graphs[i],coordinates[int(i[0])-1])
    sum_nodes+=nodes
    sum_edges+=edges
print(f"Total sum of nodes: {sum_nodes}")
print(f"Total sum of edges: {sum_edges}")

Total sum of nodes: 94
Total sum of edges: 87


In [22]:
from brute_graph_merging import merge_graphlets
sum_nodes=0
sum_edges=0
for i in blip_graphs:
    nodes,edges=merge_graphlets(blip_graphs[i],coordinates[int(i[0])-1])
    sum_nodes+=nodes
    sum_edges+=edges
print(f"Total sum of nodes: {sum_nodes}")
print(f"Total sum of edges: {sum_edges}")

Total sum of nodes: 62
Total sum of edges: 50


In [23]:
from brute_graph_merging import merge_graphlets
sum_nodes=0
sum_edges=0
for i in git_graphs:
    nodes,edges=merge_graphlets(git_graphs[i],coordinates[int(i[0])-1])
    sum_nodes+=nodes
    sum_edges+=edges
print(f"Total sum of nodes: {sum_nodes}")
print(f"Total sum of edges: {sum_edges}")

Total sum of nodes: 76
Total sum of edges: 55
